In [10]:
import tensorflow as tf
import time


#开始时间
start = time.clock()



#导入训练集
fileNameQue = tf.train.string_input_producer(["tfrecords/train.tfrecords"])
reader = tf.TFRecordReader()
key,value = reader.read(fileNameQue)
features = tf.parse_single_example(value,features={ 'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string)})

image = tf.decode_raw(features["img_raw"], tf.uint8)
#归一化
image=tf.cast(image,tf.float32)*(1/255)
image=tf.reshape(image,[48,24,3])
#图片转变为黑白图片
image=tf.split(image,3,2)[0]

label=features['label']
label = tf.cast(label, tf.int32)
init = tf.global_variables_initializer()
# 随机选取图片形成一个ｂａｔｃｈ
imageBatch, labelBatch = tf.train.shuffle_batch([image, label], batch_size = 10,capacity = 20000, min_after_dequeue = 19000)






#导入测试集
TfileNameQue = tf.train.string_input_producer(["tfrecords/validation.tfrecords"])
Treader = tf.TFRecordReader()
Tkey,Tvalue = Treader.read(TfileNameQue)
Tfeatures = tf.parse_single_example(Tvalue,features={ 'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string)})

Timage = tf.decode_raw(Tfeatures["img_raw"], tf.uint8)
Timage=tf.cast(Timage,tf.float32)*(1/255)
Timage=tf.reshape(Timage,[48,24,3])
Timage=tf.split(Timage,3,2)[0]

Tlabel=Tfeatures['label']
Tlabel = tf.cast(Tlabel, tf.int32)
init = tf.global_variables_initializer()

TimageBatch, TlabelBatch = tf.train.shuffle_batch([Timage, Tlabel], batch_size = 10,capacity = 3000, min_after_dequeue = 2900)






# 一，函数声明部分

#权重
def weight_variable(shape):
    # 正态分布，标准差为0.1，默认最大为1，最小为-1，均值为0
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

#偏差
def bias_variable(shape):
    # 创建一个结构为shape矩阵也可以说是数组shape声明其行列，初始化所有值为0.1
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#卷积
def conv2d(x, W):  
    # 卷积遍历各方向步数为1，SAME：边缘外自动补0，遍历相乘
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  

#最大池化
def max_pool_3x3(x):  
    # 池化卷积结果（conv2d）池化层采用kernel大小为2*2，步数也为2，周围补0，取最大值。数据量缩小了4倍
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],strides=[1, 3, 3, 1], padding='SAME')  

# 二，定义输入输出结构

# 声明一个占位符，None表示输入图片的数量不定，28*28图片分辨率
xs = tf.placeholder(tf.float32, [None, 48*24]) 
# 类别是0-9总共10个类别，对应输出分类结果
ys = tf.placeholder(tf.int32, [None]) 
keep_prob = tf.placeholder(tf.float32)
# x_image又把xs reshape成了48*24*1的形状，因为是灰色图片，所以通道是1.作为训练时的input，-1代表图片数量不定
x_image = tf.reshape(xs, [-1, 48, 24, 1]) 


# 三，搭建网络,定义算法公式，也就是forward时的计算

## 第一层卷积操作 ##
# 第一二参数值得卷积核尺寸大小，即patch，第三个参数是图像通道数，第四个参数是卷积核的数目，代表会出现多少个卷积特征图像;
W_conv1 = weight_variable([5, 5, 1, 32]) 
# 对于每一个卷积核都有一个对应的偏置量。
b_conv1 = bias_variable([32])  
# 图片乘以卷积核，并加上偏执量，卷积结果48x24x32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
# 池化结果16x8x32 卷积结果乘以池化卷积核
h_pool1 = max_pool_3x3(h_conv1) 

## 第二层卷积操作 ##   
# 32通道卷积，卷积出96个特征  
w_conv2 = weight_variable([5,5,32,96]) 
# 96个偏执数据
b_conv2  = bias_variable([96]) 
# 注意h_pool1是上一层的池化结果，#卷积结果16x8x96
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)  

h_pool2 = max_pool_3x3(h_conv2)  
# 原图像尺寸48*24，第一轮图像缩小为16*8，共有32张，第二轮后图像缩小为6*3，共有96张  
# 池化结果6x3x96

## 第三层全连接操作 ##
# 二维张量，第一个参数6*3*96的patch，也可以认为是只有一行6*3*96个数据的卷积，第二个参数代表卷积个数共1024个
W_fc1 = weight_variable([6*3*96, 1024]) 
# 1024个偏执数据
b_fc1 = bias_variable([1024]) 
# 将第二层卷积池化结果reshape成只有一行6*3*96个数据# [n_samples, 6, 3, 96] ->> [n_samples, 6*3*96]
h_pool2_flat = tf.reshape(h_pool2, [-1, 6*3*96]) 
# 卷积操作，结果是1*1*1024，单行乘以单列等于1*1矩阵，matmul实现最基本的矩阵相乘，不同于tf.nn.conv2d的遍历相乘，自动认为是前行向量后列向量
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) 

# dropout操作，减少过拟合，其实就是降低上一层某些输入的权重scale，甚至置为0，升高某些输入的权值，甚至置为2，防止评测曲线出现震荡，个人觉得样本较少时很必要
# 使用占位符，由dropout自动确定scale，也可以自定义，比如0.5，根据tensorflow文档可知，程序中真实使用的值为1/0.5=2，也就是某些输入乘以2，同时某些输入乘以0
# keep_prob = tf.placeholder(tf.float32) 
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob) #对卷积结果执行dropout操作

## 第四层输出操作 ##
# 二维张量，1*1024矩阵卷积，共10个卷积，对应我们开始的ys长度为10
W_fc2 = weight_variable([1024, 34])  
b_fc2 = bias_variable([34])  
# 最后的分类，结果为1*1*10 softmax和sigmoid都是基于logistic分类算法，一个是多分类一个是二分类
y_conv=tf.matmul(h_fc1_drop, W_fc2) + b_fc2
# print(y_conv.shape)
# 四，定义loss(最小误差概率)，选定优化优化loss，

cross_entropy=tf.reduce_sum(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_conv,labels=ys))
# cross_entropy=tf.reduce_sum(
#     tf.nn.softmax_cross_entropy_with_logits(logits=y_conv,labels=ys))


# cross_entropy=tf.reduce_sum(tf.argmax(y_conv,1),ys)
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy) # 调用优化器优化，其实就是通过喂数据争取cross_entropy最小化  
# print(train_step)
# # 五，开始数据训练以及评测
correct_prediction = tf.equal(tf.cast(tf.argmax(y_conv,1),tf.int32), ys)
# print(correct_prediction)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#模型保存
saver = tf.train.Saver()

# print(accuracy)
import numpy as np
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    
    #队列
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    new_dict={}
    
    for i in range(5000):
        img,lab=sess.run([imageBatch, labelBatch])
        img=np.reshape(img,[-1,1152])
        lab=np.reshape(lab,[-1])
#         print(img.shape)
#         print(lab.shape)
        
       
        if i % 100 == 0:
#            
            train_accuracy = sess.run(accuracy,
                                feed_dict={xs:img, ys: lab, keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))
            
            saver.save(sess, 'save/sa.netmodel')
        
        
        _, ce = sess.run([train_step, cross_entropy], feed_dict={xs:img, ys: lab, keep_prob: 0.5})
        if i % 100 == 0:
            print(ce)
        
        c, lab = sess.run([correct_prediction, ys], feed_dict={xs:img, ys:lab, keep_prob:1.0})

        for j in range(len(c)):
            if lab[j] in new_dict:
                if c[j] == True:
                    new_dict[lab[j]][0] += 1
                    new_dict[lab[j]][1] += 1
                else:
                    new_dict[lab[j]][1] += 1
            else:
                if c[j] == True:
                    new_dict[lab[j]] = [1, 1]
                else:
                    new_dict[lab[j]] = [0, 1]     
                              
                              
    Timg,Tlab=sess.run([TimageBatch, TlabelBatch])
    Timg=np.reshape(Timg,[-1,1152])
    Tlab=np.reshape(Tlab,[-1])
    
    test_acc = accuracy.eval(feed_dict={xs:Timg, ys: Tlab, keep_prob: 1.0})
    print("validation accuracy %g" % test_acc)
    
    for k in new_dict:
        print("标签%s的召回率是：%.3f" %(k,  new_dict[k][0]/new_dict[k][1]))
    
    #结束时间
    end = time.clock()
    print('running time is:',end-start)
    
    
    coord.request_stop()
    coord.join(threads)

step 0, training accuracy 0
125.106
step 100, training accuracy 0.2
34.2457
step 200, training accuracy 0.3
32.9542
step 300, training accuracy 0.3
31.7501
step 400, training accuracy 0.4
27.5895
step 500, training accuracy 0.8
14.6458
step 600, training accuracy 0.5
21.5096
step 700, training accuracy 0.9
13.0551
step 800, training accuracy 0.9
9.16472
step 900, training accuracy 1
4.81043
step 1000, training accuracy 1
4.34846
step 1100, training accuracy 1
4.28719
step 1200, training accuracy 1
9.08192
step 1300, training accuracy 0.9
4.69425
step 1400, training accuracy 1
0.765753
step 1500, training accuracy 1
6.20816
step 1600, training accuracy 1
5.09926
step 1700, training accuracy 1
0.414577
step 1800, training accuracy 1
1.85306
step 1900, training accuracy 0.9
5.9861
step 2000, training accuracy 1
0.218542
step 2100, training accuracy 1
1.67132
step 2200, training accuracy 1
0.585665
step 2300, training accuracy 1
0.173789
step 2400, training accuracy 0.9
2.48017
step 2500, 